**Tách riêng dư liệu chỉ số tài chính từ file du_lieu_tai_chinh**

In [6]:
import pandas as pd

# Tên file Excel của bạn
file_name = "bao_cao_tai_chinh1.xlsx"

# Tên của sheet bạn muốn đọc
sheet_to_read = "Chi_so_tai_chinh"

try:
    # Đọc sheet cụ thể từ file Excel
    df_chi_so = pd.read_excel(file_name, sheet_name=sheet_to_read)
    
    # In DataFrame ra để xem
    print(f"--- Dữ liệu từ sheet: {sheet_to_read} ---")
    print(df_chi_so)

except FileNotFoundError:
    print(f"LỖI: Không tìm thấy file '{file_name}'.")
    print("Vui lòng đảm bảo file này ở cùng thư mục với script Python của bạn.")
except ValueError as e:
    # Lỗi này xảy ra nếu tên sheet không tồn tại
    print(f"LỖI: {e}")
except Exception as e:
    print(f"Đã xảy ra lỗi không xác định: {e}")

--- Dữ liệu từ sheet: Chi_so_tai_chinh ---
     Unnamed: 0 Meta Unnamed: 2 Unnamed: 3 Chỉ tiêu cơ cấu nguồn vốn  \
0           NaN   CP        Năm         Kỳ          (Vay NH+DH)/VCSH   
1           NaN  NaN        NaN        NaN                       NaN   
2          0.00  HPG       2025          3                      0.98   
3          1.00  HPG       2025          2                      0.99   
4          2.00  HPG       2025          1                      0.99   
..          ...  ...        ...        ...                       ...   
252       46.00  TVN       2014          1                      2.43   
253       47.00  TVN       2013          4                      2.69   
254       48.00  TVN       2013          3                      2.69   
255       49.00  TVN       2013          2                      2.60   
256       50.00  TVN       2013          1                      2.81   

    Unnamed: 5      Unnamed: 6           Unnamed: 7  \
0      Nợ/VCSH  TSCĐ / Vốn CSH  Vốn C

**Lấy dữ liệu vĩ mô**

In [3]:
# Install libraries if needed
!pip install requests pandas --quiet

import requests
import pandas as pd
from functools import reduce

# ✅ Country - Only Vietnam
country = "VNM"

# ✅ Indicators required (Updated with IIP)
indicators = {
    "GDP Growth (%)": "NY.GDP.MKTP.KD.ZG",
    "CPI (Inflation, annual %)": "FP.CPI.TOTL.ZG",
    "Exchange Rate (USD/VND)": "PA.NUS.FCRF",
    "Interest Rate (%) (Lending Rate Proxy)": "FR.INR.LEND",
    "FDI Inflow (US$)": "BX.KLT.DINV.CD.WD",
    "Industrial Production Growth (IIP %)": "IP.PRD.INDX.ZG"
}

start_year = 2010
end_year = 2024

# ✅ World Bank API data fetch function
def get_wb_data(country_code, indicator_code, start, end):
    url = (
        f"https://api.worldbank.org/v2/country/{country_code}/indicator/"
        f"{indicator_code}?format=json&date={start}:{end}&per_page=2000"
    )
    r = requests.get(url)
    data = r.json()

    # Nếu không có dữ liệu
    if len(data) < 2 or not isinstance(data[1], list) or len(data[1]) == 0:
        print(f"⚠️ Warning: No data for indicator {indicator_code}")
        return pd.DataFrame()

    df = pd.DataFrame(data[1])[["date", "value"]]
    df["Year"] = df["date"].astype(int)
    df.drop(columns=["date"], inplace=True)
    df.rename(columns={"value": indicator_code}, inplace=True)
    return df

# ✅ Retrieve all indicators for Vietnam
data_frames = []
for name, code in indicators.items():
    print(f"Fetching {name} ...")
    df_temp = get_wb_data(country, code, start_year, end_year)
    if df_temp.empty:
        continue  # bỏ DataFrame rỗng
    df_temp.rename(columns={code: name}, inplace=True)
    data_frames.append(df_temp)

# --- BẮT ĐẦU CHỈNH SỬA ---
# (Đổi tên 'df' thành 'df_econ' theo yêu cầu)

# ✅ Merge into a single table, chỉ khi có DataFrame tồn tại
if data_frames:
    # Thay 'df' bằng 'df_econ'
    df_econ = reduce(lambda left, right: pd.merge(left, right, on="Year", how="outer"), data_frames)
    df_econ = df_econ.sort_values("Year").reset_index(drop=True)
else:
    # Thay 'df' bằng 'df_econ'
    df_econ = pd.DataFrame()
    print("❌ No data available for any indicator!")

# ✅ Formatting display
pd.options.display.float_format = '{:,.2f}'.format

# ✅ Export Excel nếu có dữ liệu
# Thay 'df.empty' bằng 'df_econ.empty'
if not df_econ.empty:
    output_filename = "Vietnam_Macro_Indicators_2010_2024.xlsx"
    # Thay 'df.to_excel' bằng 'df_econ.to_excel'
    df_econ.to_excel(output_filename, index=False)
    print("✅ File exported successfully:", output_filename)
else:
    print("❌ No Excel file exported, data empty.")

# Thay 'df' bằng 'df_econ' để hiển thị
df_econ

# --- KẾT THÚC CHỈNH SỬA ---


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Fetching GDP Growth (%) ...
Fetching CPI (Inflation, annual %) ...
Fetching Exchange Rate (USD/VND) ...
Fetching Interest Rate (%) (Lending Rate Proxy) ...
Fetching FDI Inflow (US$) ...
Fetching Industrial Production Growth (IIP %) ...
⚠️ Warning: No data for indicator IP.PRD.INDX.ZG
✅ File exported successfully: Vietnam_Macro_Indicators_2010_2024.xlsx


,GDP Growth (%),Year,"CPI (Inflation, annual %)",Exchange Rate (USD/VND),Interest Rate (%) (Lending Rate Proxy),FDI Inflow (US$)
0,6.42,2010,9.21,"18,612.92",13.14,8000000000
1,6.41,2011,18.68,"20,509.75",16.95,7430000000
2,5.50,2012,9.09,"20,828.00",13.47,8368000000
3,5.55,2013,6.59,"20,933.42",10.37,8900000000
4,6.42,2014,4.08,"21,148.00",8.66,9200000000
5,6.99,2015,0.63,"21,697.57",7.12,11800000000
6,6.69,2016,2.67,"21,935.00",6.96,12600000000
7,6.94,2017,3.52,"22,370.09",7.07,14100000000
8,7.47,2018,3.54,"22,602.05",7.37,15500000000
9,7.36,2019,2.80,"23,050.24",7.71,16120000000


**Lấy dữ liệu giá lịch sử thép**

In [1]:
import pandas as pd

# Tên file Excel của bạn
file_name = "gia_lich_su_thep1.xlsx"

# Tên của sheet bạn muốn đọc
HPG = "HPG"
NKG = "NKG"
HSG = "HSG"
POM = "POM"
TVN = "TVN"

# 1. Gộp các tên sheet vào một danh sách (list)
sheet_list = [HPG, NKG, HSG, POM, TVN]

# 2. Tạo một danh sách rỗng để chứa các DataFrame
all_data_list = []

print(f"Bắt đầu đọc file Excel: '{file_name}'...")

try:
    # 3. Lặp qua từng tên sheet trong danh sách
    for sheet_name in sheet_list:
        print(f"  -> Đang đọc sheet: {sheet_name}...")
        
        # Đọc sheet hiện tại vào DataFrame
        df = pd.read_excel(file_name, sheet_name=sheet_name)
        
        # Thêm cột 'symbol' để biết dữ liệu này của mã nào
        df['symbol'] = sheet_name
        
        # Thêm DataFrame vào danh sách
        all_data_list.append(df)

    # 4. Gộp (concatenate) tất cả DataFrame trong danh sách
    if all_data_list:
        df_merged = pd.concat(all_data_list, ignore_index=True)
        
        print("\n--- HOÀN TẤT GỘP DỮ LIỆU ---")
        print("DataFrame gộp (df_merged) đã được tạo:")
        print(df_merged)
    else:
        print("Không có dữ liệu nào để gộp.")

except FileNotFoundError:
    print(f"\nLỖI: Không tìm thấy file '{file_name}'.")
except ValueError as e:
    # Lỗi này có thể xảy ra nếu một trong các sheet không tồn tại
    print(f"\nLỖI: {e}. (Hãy kiểm tra lại tên sheet).")
except Exception as e:
    # Cần cài đặt openpyxl để đọc .xlsx
    if "Missing optional dependency 'openpyxl'" in str(e):
        print("\nLỖI: Cần cài đặt 'openpyxl'.")
        print("Hãy chạy lệnh sau: pip install openpyxl")
    else:
        print(f"\nĐã xảy ra lỗi: {e}")

Bắt đầu đọc file Excel: 'gia_lich_su_thep1.xlsx'...
  -> Đang đọc sheet: HPG...
  -> Đang đọc sheet: NKG...
  -> Đang đọc sheet: HSG...
  -> Đang đọc sheet: POM...
  -> Đang đọc sheet: TVN...

--- HOÀN TẤT GỘP DỮ LIỆU ---
DataFrame gộp (df_merged) đã được tạo:
           time   open   high    low  close    volume symbol
0    2021-10-28  33.20  33.37  32.85  33.25  21923800    HPG
1    2021-10-29  33.25  33.31  32.74  32.74  23878000    HPG
2    2021-11-01  32.74  32.74  31.93  31.93  45218500    HPG
3    2021-11-02  31.82  32.34  31.65  31.82  25669500    HPG
4    2021-11-03  32.22  32.28  31.70  31.88  29867100    HPG
...         ...    ...    ...    ...    ...       ...    ...
4920 2025-10-03   8.10   8.10   7.90   8.00    790400    TVN
4921 2025-10-06   8.00   8.40   8.00   8.30    272200    TVN
4922 2025-10-07   8.30   8.30   8.00   8.10    212800    TVN
4923 2025-10-08   8.20   8.30   8.10   8.20    246100    TVN
4924 2025-10-09   8.20   8.20   8.10   8.20    228500    TVN

[4925 

**Lấy dữ liệu giá nguyên liệu**

In [5]:
import pandas as pd

# Tên file Excel của bạn
file_name = "gia_nguyen_lieu.xlsx"

# Tên của sheet bạn muốn đọc
bot_sat = "gia_bot_sat"
than_coc = "gia_than_coc"
cot_thep = "gia_cot_thep"


# 1. Gộp các tên sheet vào một danh sách (list)
sheet_list = [bot_sat, than_coc, cot_thep]

# 2. Tạo một danh sách rỗng để chứa các DataFrame
all_data_list = []

print(f"Bắt đầu đọc file Excel: '{file_name}'...")

try:
    # 3. Lặp qua từng tên sheet trong danh sách
    for sheet_name in sheet_list:
        print(f"  -> Đang đọc sheet: {sheet_name}...")
        
        # Đọc sheet hiện tại vào DataFrame
        df = pd.read_excel(file_name, sheet_name=sheet_name)
        
        # Thêm cột 'loai_nguyen_lieu' để biết dữ liệu này của loại nào
        df['loai_nguyen_lieu'] = sheet_name
        
        # Thêm DataFrame vào danh sách
        all_data_list.append(df)

    # 4. Gộp (concatenate) tất cả DataFrame trong danh sách
    if all_data_list:
        df_merged_materials = pd.concat(all_data_list, ignore_index=True)
        
        print("\n--- HOÀN TẤT GỘP DỮ LIỆU ---")
        print("DataFrame gộp (df_merged_materials) đã được tạo:")
        print(df_merged_materials)
    else:
        print("Không có dữ liệu nào để gộp.")

except FileNotFoundError:
    print(f"\nLỖI: Không tìm thấy file '{file_name}'.")
except ValueError as e:
    # Lỗi này có thể xảy ra nếu một trong các sheet không tồn tại
    print(f"\nLỖI: {e}. (Hãy kiểm tra lại tên sheet).")
except Exception as e:
    # Kiểm tra xem có phải lỗi do thiếu 'openpyxl' không
    if "Missing optional dependency 'openpyxl'" in str(e):
        print("\nLỖI: Cần cài đặt 'openpyxl' để đọc file .xlsx.")
        print("Hãy chạy lệnh sau trong terminal: pip install openpyxl")
    else:
        print(f"\nĐã xảy ra lỗi không xác định: {e}")

Bắt đầu đọc file Excel: 'gia_nguyen_lieu.xlsx'...
  -> Đang đọc sheet: gia_bot_sat...
  -> Đang đọc sheet: gia_than_coc...
  -> Đang đọc sheet: gia_cot_thep...

--- HOÀN TẤT GỘP DỮ LIỆU ---
DataFrame gộp (df_merged_materials) đã được tạo:
          Ngày  Đơn vị: USD  % Thay đổi      Lần cuối    1   USD   =  26301.5  \
0   2025-11-01       103.34     -0.0235  2.717997e+06  vnd  /tấn NaN      NaN   
1   2025-10-01       105.83      0.0051  2.783488e+06  vnd  /tấn NaN      NaN   
2   2025-09-01       105.29      0.0342  2.769285e+06  vnd  /tấn NaN      NaN   
3   2025-08-01       101.81      0.0271  2.677756e+06  vnd  /tấn NaN      NaN   
4   2025-07-01        99.12      0.0492  2.607005e+06  vnd  /tấn NaN      NaN   
..         ...          ...         ...           ...  ...   ...  ..      ...   
462 2016-04-01       454.00      0.2507  1.194088e+07  vnd  /tấn NaN      NaN   
463 2016-03-01       363.00      0.1386  9.547444e+06  vnd  /tấn NaN      NaN   
464 2016-02-01       318.81     